In [1]:
import numpy as np

In [2]:
class NaiveBayes:

  def fit(self, X, y):
    n_samples , n_features = X.shape
    self._classes = np.unique(y)
    n_classes = len(self._classes)

    self._mean = np.zeros((n_classes,n_features), dtype=np.float64)
    self._var = np.zeros((n_classes,n_features), dtype=np.float64)
    self._priors = np.zeros(n_classes, dtype=np.float64)

    for idx, c in enumerate(self._classes):
      X_c = X[y==c]
      self._mean[idx, :] = X_c.mean(axis=0)
      self._var[idx, :] = X_c.var(axis=0)
      self._priors[idx] = X_c.shape[0]/float(n_samples)

  def predict(self,X):
    y_pred = [self._predict(x) for x in X]
    return np.array(y_pred)
  
  def _predict(self, x):
    posteriors = []

    for idx, c in enumerate(self._classes):
      prior = np.log(self._priors[idx])
      posterior = np.sum(np.log(self._pdf(idx,x)))
      posterior = prior + posterior
      posteriors.append(posterior)

    return self._classes[np.argmax(posteriors)]

  def _pdf(self, class_idx, x):
    mean = self._mean[class_idx]
    var = self._var[class_idx]
    numerator = np.exp(- (x-mean)**2 / (2 * var))
    denominator = np.sqrt(2 * np.pi * var)
    return numerator / denominator
  


In [3]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

In [4]:
def accuracy(y_true, y_pred):
  accuracy = np.sum(y_true==y_pred) / len(y_true)
  return accuracy*100

In [5]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [7]:
nb = NaiveBayes()
nb.fit(X_train,y_train)
predictions = nb.predict(X_test)

print('Naive Bayes accuracy on iris:', accuracy(y_test,predictions))

Naive Bayes accuracy on iris: 96.66666666666667


In [8]:
predictions = nb.predict(X_train)

print('Naive Bayes accuracy in training:', accuracy(y_train,predictions))

Naive Bayes accuracy in training: 96.66666666666667


In [9]:
from sklearn import metrics
expected = y_test
predicted = nb.predict(X_test)
print(metrics.classification_report(expected,predicted))
print(metrics.confusion_matrix(expected,predicted))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.86      1.00      0.92         6
           2       1.00      0.91      0.95        11

    accuracy                           0.97        30
   macro avg       0.95      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30

[[13  0  0]
 [ 0  6  0]
 [ 0  1 10]]


In [10]:
expected = y_train
predicted = nb.predict(X_train)
print(metrics.classification_report(expected,predicted))
print(metrics.confusion_matrix(expected,predicted))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        37
           1       0.95      0.95      0.95        44
           2       0.95      0.95      0.95        39

    accuracy                           0.97       120
   macro avg       0.97      0.97      0.97       120
weighted avg       0.97      0.97      0.97       120

[[37  0  0]
 [ 0 42  2]
 [ 0  2 37]]
